In [2]:
# 因为在外面没时间，所以就只是简单过了一遍教程
# 参考自：https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.24.1cd81b43uSLnlN&postId=95535


import pandas as pd
import numpy as np
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
%matplotlib inline

import itertools
import matplotlib.gridspec as gridspec
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
# from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
# from mlxtend.plotting import plot_learning_curves
# from mlxtend.plotting import plot_decision_regions

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.decomposition import PCA,FastICA,FactorAnalysis,SparsePCA

import lightgbm as lgb
# import xgboost as xgb
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [68]:
def build_model_lr(x_train,y_train):
    reg_model = linear_model.LinearRegression()
    reg_model.fit(x_train,y_train)
    return reg_model

def build_model_ridge(x_train,y_train):
    reg_model = linear_model.Ridge(alpha=0.8)#alphas=range(1,100,5)
    reg_model.fit(x_train,y_train)
    return reg_model

def build_model_lasso(x_train,y_train):
    reg_model = linear_model.LassoCV()
    reg_model.fit(x_train,y_train)
    return reg_model

def build_model_gbdt(x_train,y_train):
    estimator =GradientBoostingRegressor(loss='ls',subsample= 0.85,max_depth= 5,n_estimators = 100, learning_rate = 0.05)
    # param_grid = {  'learning_rate': [0.05,0.08,0.1,0.2],}
    # gbdt = GridSearchCV(estimator, param_grid,cv=3)
    estimator.fit(x_train,y_train)
    # print(estimator.best_params_)
    # print(gbdt.best_estimator_ )
    return estimator

'''def build_model_xgb(x_train,y_train):
    model = xgb.XGBRegressor(n_estimators=120, learning_rate=0.08, gamma=0, subsample=0.8,\
        colsample_bytree=0.9, max_depth=5) #, objective ='reg:squarederror'
    model.fit(x_train, y_train)
    return model'''

def build_model_lgb(x_train,y_train):
    estimator = lgb.LGBMRegressor(num_leaves=63,n_estimators = 1000, learning_rate = 0.01)
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1],
    }
    # gbm = GridSearchCV(estimator, param_grid)
    estimator.fit(x_train, y_train)
    return estimator

In [113]:
train_data_tree = pd.read_csv('train_tree.csv', sep = ' ')
test_data_tree = pd.read_csv('test_tree.csv', sep = ' ')
train_data_lr = pd.read_csv('train_lr.csv', sep = ' ')
test_data_lr = pd.read_csv('test_lr.csv', sep = ' ')
test_data_tree = test_data_tree.drop('price', axis = 1)
test_data_lr = test_data_lr.drop('price', axis = 1)

In [114]:
train_data_tree_label = train_data_tree['price']
train_data_lr_label = train_data_lr['price']
train_data_tree = train_data_tree.drop('price', axis = 1)
train_data_lr = train_data_lr.drop('price', axis = 1)

In [115]:
# copy from https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.9.1cd81b43uSLnlN&postId=95460
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum()
    print('内存占用{:.2f} MB'.format(start_mem/1024/1024))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum()
    print('优化后内存为: {:.2f} MB'.format(end_mem/1024/1024))
    print('内存使用减少 {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [116]:
train_data_lr = reduce_mem_usage(train_data_lr)

内存占用34.33 MB
优化后内存为: 8.87 MB
内存使用减少 74.2%


In [117]:
train_data_tree = reduce_mem_usage(train_data_tree)

内存占用96.13 MB
优化后内存为: 24.75 MB
内存使用减少 74.3%


In [118]:
## Split data with val
x_train_tree,x_val_tree,y_train_tree,y_val_tree = train_test_split(train_data_tree,train_data_tree_label,test_size=0.3)
x_train_lr,x_val_lr,y_train_lr,y_val_lr = train_test_split(train_data_lr,train_data_lr_label,test_size=0.3)
## Train and Predict


print('Predict Ridge...')
model_ridge = build_model_ridge(x_train_lr,y_train_lr)
val_ridge = model_ridge.predict(x_val_lr)
subA_ridge = model_ridge.predict(test_data_lr)

print('Predict Lasso...')
model_lasso = build_model_lasso(x_train_lr,y_train_lr)
val_lasso = model_lasso.predict(x_val_lr)
subA_lasso = model_lasso.predict(test_data_lr)

print('Predict GBDT...')
model_gbdt = build_model_gbdt(x_train_lr,y_train_lr)
val_gbdt = model_gbdt.predict(x_val_lr)
subA_gbdt = model_gbdt.predict(test_data_lr)

print('predict lgb...')
model_lgb = build_model_lgb(x_train_lr,y_train_lr)
val_lgb = model_lgb.predict(x_val_lr)
subA_lgb = model_lgb.predict(test_data_lr)

Predict Ridge...
Predict Lasso...
Predict GBDT...
predict lgb...


In [61]:
def Sta_inf(data):
    print('_min',np.min(data))
    print('_max:',np.max(data))
    print('_mean',np.mean(data))
    print('_ptp',np.ptp(data))
    print('_std',np.std(data))
    print('_var',np.var(data))

In [119]:
def Weighted_method(test_pre1,test_pre2,test_pre3,w=[1/3,1/3,1/3]):
    Weighted_result = w[0]*pd.Series(test_pre1)+w[1]*pd.Series(test_pre2)+w[2]*pd.Series(test_pre3)
    return Weighted_result

## Init the Weight
w = [0.6,0.1,0.3]

## 测试验证集准确度
val_pre = Weighted_method(val_lgb,val_lasso,val_gbdt,w)
MAE_Weighted = mean_absolute_error(y_val_lr,val_pre)
print('MAE of Weighted of val:',MAE_Weighted)

## 预测数据部分
subA = Weighted_method(subA_lgb,subA_lasso,subA_gbdt,w)
print('Sta inf:')
Sta_inf(subA)
## 生成提交文件
sub = pd.DataFrame()
sub['SaleID'] = test_data_lr.index
sub['price'] = subA
sub.to_csv('./sub_Weighted.csv',index=False)

MAE of Weighted of val: 816.2918858623362
Sta inf:
_min 242.89383685441936
_max: 83479.0171353402
_mean 5908.597606268312
_ptp 83236.12329848578
_std 6696.181251674079
_var 44838843.355271436


In [121]:
# 与lgb进行对比
val_lgb_pred = model_lgb.predict(x_val_lr)
MAE_lgb = mean_absolute_error(y_val_lr,val_lgb_pred)
print('MAE of lgb:',MAE_lgb)
# 哈哈哈哈还不如单独的lgb，看来是因为lasso太菜了，然后gbdt应该跟lgbm的模型差异性不够大

MAE of lgb: 624.9831074054401


In [122]:
## Starking

## 第一层
train_lgb_pred = model_lgb.predict(x_train_lr)
train_lasso_pred = model_lasso.predict(x_train_lr)
train_gbdt_pred = model_gbdt.predict(x_train_lr)

Strak_X_train = pd.DataFrame()
Strak_X_train['Method_1'] = train_lgb_pred
Strak_X_train['Method_2'] = train_lasso_pred
Strak_X_train['Method_3'] = train_gbdt_pred

Strak_X_val = pd.DataFrame()
Strak_X_val['Method_1'] = val_lgb
Strak_X_val['Method_2'] = val_lasso
Strak_X_val['Method_3'] = val_gbdt

Strak_X_test = pd.DataFrame()
Strak_X_test['Method_1'] = subA_lgb
Strak_X_test['Method_2'] = subA_lasso
Strak_X_test['Method_3'] = subA_gbdt

In [124]:
Strak_X_test.head()

,Method_1,Method_2,Method_3
0,1287.967308,4857.317401,1286.867735
1,1829.771310,3969.656694,1834.269263
2,8617.967186,7757.138913,8266.451319
3,1298.218316,4828.111254,1193.124838
4,2089.220179,5770.743187,2106.774572


In [123]:
## level2-method 
model_lr_Stacking = build_model_lr(Strak_X_train,y_train_lr)
## 训练集
train_pre_Stacking = model_lr_Stacking.predict(Strak_X_train)
print('MAE of Stacking-LR:',mean_absolute_error(y_train,train_pre_Stacking))

## 验证集
val_pre_Stacking = model_lr_Stacking.predict(Strak_X_val)
print('MAE of Stacking-LR:',mean_absolute_error(y_val_lr,val_pre_Stacking))

## 预测集
print('Predict Stacking-LR...')
subA_Stacking = model_lr_Stacking.predict(Strak_X_test)


MAE of Stacking-LR: 5925.4080037060885
MAE of Stacking-LR: 618.7730740849673
Predict Stacking-LR...
